In [27]:
from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    MetricsAggregationFn,
    NDArrays,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
import flwr as fl
import pandas as pd
from typing import Callable, Dict, List, Optional, Tuple, Union
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from flwr.server.strategy.aggregate import aggregate
from flwr.server.client_proxy import ClientProxy

In [28]:
# https://drive.google.com/drive/u/1/folders/1-AbVA2945Ep1IadGEnXa9AjeDFwh-Bwl

In [29]:
NUM_FEATURE = 256
NUM_CLASSES = 3
PACKET_NUM = 20
client_lr = 3e-4
NUM_EPOCHS = 30
BATCH_SIZE = 16

In [30]:
def most_frequent(List):
    return max(set(List), key=List.count)


def data_processing(df, NUM_FEATURES):
   y_train = df['Label']
   flow_id = df['flow_id']

   df = df/255

   X_train = df.drop(['Label', 'flow_id'], axis=1)
   X_train = X_train.to_numpy()

   X_train = X_train.reshape(-1, 20, NUM_FEATURES)
   y_train = y_train.to_numpy()

   y_train = y_train.reshape(-1, 20)[:, -1]
   return X_train, y_train

In [31]:
import pandas as pd
import numpy as np
# Đọc dữ liệu từ file feather
diff_1_distribution = pd.read_feather(
    './Bản sao của A_data1.feather')
diff_2_distribution = pd.read_feather(
    './Bản sao của A_data2.feather')
train_distribution = pd.read_feather(
    './df0.09.feather')
eval_distribution = pd.read_feather(
    './df0.91.feather')

x_train1, y_train1 = data_processing(
    diff_1_distribution, NUM_FEATURE)
x_train2, y_train2 = data_processing(
    diff_2_distribution, NUM_FEATURE)
x_train3, y_train3 = data_processing(
    train_distribution, NUM_FEATURE)
x_test, y_test = data_processing(
    eval_distribution, NUM_FEATURE)

x_train1, y_train1 = np.concatenate((x_train1, x_train3)), np.concatenate((y_train1, y_train3))
x_train2, y_train2 = np.concatenate((x_train2, x_train3)), np.concatenate((y_train2, y_train3))
list_x_train = [x_train1, x_train2]
list_y_train = [y_train1, y_train2]


In [32]:
from flwr.server.strategy.fedavg import FedAvg
class FedImp(FedAvg):
    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
    
        weights_results = [
                (parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples*np.exp(fit_res.metrics["er"]))
            
                for _, fit_res in results
            ]
        aggregated_ndarrays = aggregate(weights_results)
        parameters_aggregated = ndarrays_to_parameters(aggregated_ndarrays)
        metrics_aggregated = {}
        return parameters_aggregated, metrics_aggregated

In [33]:
# Định nghĩa mô hình
def get_model():
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='Same',
                     activation='relu', input_shape=(20, NUM_FEATURE, 1)))
    model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='Same',
                     activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same',
                     activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same',
                     activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_CLASSES, activation="softmax"))

    model.compile(optimizer=tf.keras.optimizers.Adam(
        learning_rate=client_lr), loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

    return model

class Client(fl.client.NumPyClient):
    def __init__(self, model, x_train, y_train) -> None:
        self.model = model
        self.x_train, self.y_train = x_train, y_train

    def get_parameters(self, config):
        return self.model.get_weights()
    
    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.x_train, self.y_train,verbose=2, batch_size=BATCH_SIZE, epochs=1)
        return self.model.get_weights(), len(self.x_train), {"er" : 1}
    
    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, accuracy = self.model.evaluate(self.x_train, self.y_train)
        return loss, len(self.x_train), {'accuracy': accuracy}

def client_fn(cid: str) -> fl.client.Client:
    model = get_model()
    x_train_cid = list_x_train[int(cid)]
    y_train_cid = list_y_train[int(cid)]
    return Client(model, x_train_cid, y_train_cid)

def evaluate_DNN_CL(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    net = get_model()
    net.set_weights(parameters)
    loss, accuracy = net.evaluate(x_test, y_test, verbose=2)
    predictions = net.predict(x_test, verbose = 3)  # Dự đoán nhãn trên tập kiểm tra
    y_pred = np.argmax(predictions, axis=1)
    precision = precision_score(y_pred, y_test, average='weighted',zero_division = 0)
    recall = recall_score(y_pred, y_test, average='weighted',zero_division = 0)
    f1score = f1_score(y_pred, y_test, average='weighted',zero_division = 0)
    return loss, {"accuracy": accuracy,"precision": precision,"recall": recall,"f1score": f1score}

strategy = FedImp(evaluate_fn=evaluate_DNN_CL, fraction_evaluate=0)

h = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=2,
    strategy=strategy,
    config=fl.server.ServerConfig(num_rounds=10),
    client_resources = {'num_cpus': 8, 'num_gpus': 0},
)

INFO flwr 2024-06-23 19:39:41,161 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2024-06-23 19:39:46,852	INFO worker.py:1724 -- Started a local Ray instance.
INFO flwr 2024-06-23 19:39:47,806 | app.py:213 | Flower VCE: Ray initialized with resources: {'memory': 75056505856.0, 'object_store_memory': 36452788224.0, 'CPU': 16.0, 'node:192.168.39.172': 1.0, 'node:__internal_head__': 1.0}
INFO flwr 2024-06-23 19:39:47,808 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-06-23 19:39:47,812 | app.py:242 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 8, 'num_gpus': 0}
INFO flwr 2024-06-23 19:39:47,834 | app.py:288 | Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO flwr 2024-06-23 19:39:47,836 | server.py:89 | Initializing global parameters
INFO flwr 2024-06-23 19:39:47,839 | server.py:276 | Requesting initial parameters from

254/254 - 5s - loss: 1.0811 - sparse_categorical_accuracy: 0.5233 - 5s/epoch - 20ms/step


(raylet) [2024-06-23 19:40:05,958 E 72039 72072] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-23_19-39-42_969375_64620 is over 95% full, available space: 2487750656; capacity: 51989970944. Object creation will fail if spilling is required.
INFO flwr 2024-06-23 19:40:06,131 | server.py:94 | initial parameters (loss, other metrics): 1.081117033958435, {'accuracy': 0.5232744812965393, 'precision': 0.9998765279664156, 'recall': 0.5232744783306581, 'f1score': 0.687009845253189}
INFO flwr 2024-06-23 19:40:06,131 | server.py:104 | FL starting
DEBUG flwr 2024-06-23 19:40:06,132 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 2)
(DefaultActor pid=72868) /home/bkcs/anaconda3/lib/python3.11/site-packages/ray/_private/pydantic_compat.py:2: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
(DefaultActor pid=72868)   from pkg_resources import packaging
(DefaultActor pid=72868) /home/bkcs/anaconda3

(DefaultActor pid=72868) 441/441 - 123s - loss: 0.2873 - sparse_categorical_accuracy: 0.8678 - 123s/epoch - 280ms/step


(raylet) [2024-06-23 19:42:17,457 E 72039 72072] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-23_19-39-42_969375_64620 is over 95% full, available space: 2487566336; capacity: 51989970944. Object creation will fail if spilling is required.
DEBUG flwr 2024-06-23 19:42:20,792 | server.py:236 | fit_round 1 received 2 results and 0 failures
(raylet) [2024-06-23 19:42:27,457 E 72039 72072] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-23_19-39-42_969375_64620 is over 95% full, available space: 2487562240; capacity: 51989970944. Object creation will fail if spilling is required.


...

(DefaultActor pid=72869) 540/540 - 125s - loss: 0.0557 - sparse_categorical_accuracy: 0.9800 - 125s/epoch - 231ms/step


(raylet) [2024-06-23 20:04:28,837 E 72039 72072] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-23_19-39-42_969375_64620 is over 95% full, available space: 2485805056; capacity: 51989970944. Object creation will fail if spilling is required.


254/254 - 8s - loss: 0.1134 - sparse_categorical_accuracy: 0.9641 - 8s/epoch - 31ms/step


(raylet) [2024-06-23 20:04:38,951 E 72039 72072] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-23_19-39-42_969375_64620 is over 95% full, available space: 2485805056; capacity: 51989970944. Object creation will fail if spilling is required.
INFO flwr 2024-06-23 20:04:43,519 | server.py:125 | fit progress: (10, 0.11339615285396576, {'accuracy': 0.964069664478302, 'precision': 0.9642113114417025, 'recall': 0.9640696382269416, 'f1score': 0.9641089038754347}, 1477.3867283270083)
INFO flwr 2024-06-23 20:04:43,519 | server.py:171 | evaluate_round 10: no clients selected, cancel
INFO flwr 2024-06-23 20:04:43,520 | server.py:153 | FL finished in 1477.3879834140098
INFO flwr 2024-06-23 20:04:43,520 | app.py:226 | app_fit: losses_distributed []
INFO flwr 2024-06-23 20:04:43,521 | app.py:227 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-06-23 20:04:43,521 | app.py:228 | app_fit: metrics_distributed {}
INFO flwr 2024-06-23 20:04:43,521 | app.py:229 | app_fit: losses_centrali

(raylet) [2024-06-23 20:04:48,960 E 72039 72072] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-23_19-39-42_969375_64620 is over 95% full, available space: 2485743616; capacity: 51989970944. Object creation will fail if spilling is required.
(raylet) [2024-06-23 20:04:58,976 E 72039 72072] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-23_19-39-42_969375_64620 is over 95% full, available space: 2485727232; capacity: 51989970944. Object creation will fail if spilling is required.
(raylet) [2024-06-23 20:05:08,993 E 72039 72072] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-23_19-39-42_969375_64620 is over 95% full, available space: 2485723136; capacity: 51989970944. Object creation will fail if spilling is required.
(raylet) [2024-06-23 20:05:19,011 E 72039 72072] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-23_19-39-42_969375_64620 is over 95% full, available space: 2485723136; capacity: 51989970944. Object creation will fa